In [169]:
import numpy as np
from typing import Iterable, NamedTuple

In [3]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

In [4]:
Vector = NamedTuple('Vector', [('x', np.float32), ('y', np.float32), ('z', np.float32)])

def vec(v: Vector) -> np.array:
    return np.array(v).reshape((3, 1))

def norm(v: Vector) -> np.array:
    return np.linalg.norm(vec(v), 2, 0)

def normalize(v: Vector) -> np.array:
    v = vec(v)
    return v / norm(v)

Let's define a `Pose` base class holding both the position and orientation of an object.

In [5]:
class Pose:
    def __init__(self, pos: Vector):
        self.pos = vec(pos)  # x,y,z
        self.rot = np.matrix([[1, 0, 0], 
                              [0, 1, 0], 
                              [0, 0, 1]], dtype=np.float32)
        
    def lookat(self, pos: Vector, up: Vector=(0, 0, 1)):
        v = normalize(vec(pos) - self.pos)
        right = normalize(np.cross(normalize(up), v, axis=0))       
        up = normalize(np.cross(v, right, axis=0))
        self.rot = np.matrix(np.hstack([v, right, up]))

p = Pose((0, 0, 1))
p.lookat((5, 0, 0))
print(p.rot)

[[ 0.98058068 -0.          0.19611614]
 [ 0.          1.          0.        ]
 [-0.19611614  0.          0.98058068]]


Next, we define a sensor:

In [6]:
class Sensor(Pose):
    def __init__(self, pos: Vector=(0, 0, 0)):
        super().__init__(pos)

And finally a Lighthouse base station:

In [183]:
RelativePosition = NamedTuple('RelativePosition', [('h', float), ('v', float), 
                                                   ('dist', float), ('visible', bool)])
SweepHitTime = NamedTuple('SweepHitTime', [('h', float), ('v', float), ('rel_pos', RelativePosition)])

class Lighthouse(Pose):
    def __init__(self, pos: Vector=(0, 0, 1)):
        super().__init__(pos)
        self.x = vec(Vector(1, 0, 0))
        self.y = vec(Vector(0, 1, 0))
        self.z = vec(Vector(0, 0, 1))
        self.update_rate = 60  # Hz
        self.omega = self.update_rate * (360. * np.pi/180.)  # rad/s
    
    def sweep(self, sensors: Iterable[Sensor]):
        timings = []  # type: List[SweepHitTime]
        for s in sensors:
            # TODO: The sensor can only be seen if it's normal vector is pointing towards the Lighthouse
            rel_pos = self.get_angle(s.pos)
            t_h = rel_pos.h / self.omega
            t_v = rel_pos.v / self.omega
            timings.append(SweepHitTime(h=t_h, v=t_v, rel_pos=rel_pos))
        return timings
            
    def get_angle(self, vec: np.array) -> RelativePosition:
        # Obtain direction vector in Lighthouse coordinates
        vec = self.rot.T * (vec - self.pos)
        dist = norm(vec)
        vec = vec / dist
        
        # Obtain horizontal angle (in Lighthouse coordinates)
        cross_h = np.cross(vec, self.z, axis=0)
        assert cross_h[0] == np.dot(vec.T, self.y), 'Cross product assumption failed'
        angle_h = np.arccos(cross_h[0])
        in_front_h = cross_h[1] <= 0
        
        # Obtain vertical angle (in Lighthouse coordinates)
        cross_v = np.cross(self.y, vec, axis=0)
        assert cross_v[0] == np.dot(vec.T, self.z), 'Cross product assumption failed'
        angle_v = np.arccos(cross_v[0])
        in_front_v = cross_v[2] <= 0
        
        assert in_front_h == in_front_v, 'Plane side assertion failed'
        return RelativePosition(h=angle_h[0], v=angle_v[0], 
                                dist=dist[0], visible=in_front_h[0])
        
        


l = Lighthouse((0, 0, 2.3))
l.lookat((2.3, 0, 0))
l.rot

matrix([[ 0.70710678, -0.        ,  0.70710678],
        [ 0.        ,  1.        ,  0.        ],
        [-0.70710678,  0.        ,  0.70710678]])

In [188]:
center_x =  2.9
center_y = -.7
center_z =  .8

l.sweep([
    Sensor(( .1 + center_x,  .1 + center_y, center_z)),
    Sensor(( .1 + center_x, -.1 + center_y, center_z)),
    Sensor((-.1 + center_x, -.1 + center_y, center_z)),
    Sensor((-.1 + center_x,  .1 + center_y, center_z))
])

[SweepHitTime(h=0.004636208618359907, v=0.0033270012858700035, rel_pos=RelativePosition(h=1.7478094723139002, v=1.2542499357807597, dist=3.4073450074801634, visible=True)),
 SweepHitTime(h=0.004787740612933439, v=0.0033372864727156397, rel_pos=RelativePosition(h=1.804935688426222, v=1.2581273598729656, dist=3.448187929913333, visible=True)),
 SweepHitTime(h=0.00482111289496012, v=0.0034121484930216446, rel_pos=RelativePosition(h=1.8175167423520477, v=1.2863496766361138, dist=3.2756678708318394, visible=True)),
 SweepHitTime(h=0.004661874667497884, v=0.003401821148964252, rel_pos=RelativePosition(h=1.7574853448841254, v=1.282456359649498, dist=3.232645975048922, visible=True))]